In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.cluster import DBSCAN, KMeans
from IPython.display import display, HTML

import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

home_path = "/home/arasyildiz/Projects/invent_analytics/"
store_types_path = home_path + "store_types.csv"
product_types_path = home_path + "product_types.csv"
assignment1a_path = home_path + "assignment4.1a.csv"
assignment1b_path = home_path + "assignment4.1b.csv"
assignment1c_path = home_path + "assignment4.1c.csv"
promotions_path = home_path + "PromotionDates.csv"

sales1a = pd.read_csv(assignment1a_path)
features1c = pd.read_csv(assignment1c_path)
sales1b = pd.read_csv(assignment1b_path)
all_promotions = pd.read_csv(promotions_path)
product_types = pd.read_csv(product_types_path).iloc[:,1:]
store_types = pd.read_csv(store_types_path).iloc[:,1:]


In [2]:
product_features = product_types.merge(features1c, how='inner', on='ProductCode')
features = product_features.merge(store_types, how='cross').set_index(['ProductCode', 'StoreCode'])
print(features.head())

                       P_type ProductGroup1  ProductGroup2  S_type
ProductCode StoreCode                                             
1           1          medium             A              5  medium
            2          medium             A              5    fast
            3          medium             A              5    slow
            4          medium             A              5  medium
            5          medium             A              5    slow


In [3]:
def promo_and_non_promo(sales, promotions):
    #Printing the shape of sales csv
    print(f'- {sales.shape[0]} rows, {sales.shape[1]} columns')
       
    # # of stores and # of products
    n_product, n_store = len(sales['ProductCode'].unique()), len(sales['StoreCode'].unique())
    print(f"- number of stores: {n_store}, number of product: {n_product}\n")

    #add date variables iso.week,
    sales["DateTime"] = pd.to_datetime(sales['Date'])
    sales["Week"] = sales['DateTime'].dt.isocalendar().week
    n_week = len(sales["Week"].unique())

    # mask for non-promotion times
    promotions['StartDate'] = pd.to_datetime(promotions['StartDate'],format="%m/%d/%Y")
    promotions['EndDate'] = pd.to_datetime(promotions['EndDate'])
    
    print(promotions['StartDate'].dtype == 'datetime64[ns]')
    print(promotions['EndDate'].dtype == 'datetime64[ns]')
    print('hoop')
    
    print("Promotion Dates")
    print(pd.concat([promotions['StartDate'].dt.isocalendar().week, promotions['EndDate'].dt.isocalendar().week,
                     promotions['StartDate'].dt.day_name(), promotions['EndDate'].dt.day_name()], axis=1).to_string(index=False))
    
    #print("While calculating the weekly averages(see below) it would better to disregard week 7, 12, 22, 26 \
    #since most of the days in those weeks are on promotion.\n")

    sales["Time"] = "Non-Promo"
    for i, p in promotions.iterrows():
        sales.loc[(p.StartDate <= sales['DateTime']) & (sales['DateTime'] <= p.EndDate), "Time"] = p.Period

    no_promotion = sales["Time"] == "Non-Promo"
    non_promo_sales = sales[no_promotion]
    promo_sales = sales[~no_promotion]
    
    print("\nPromo vs Non Promo Time")
    print("# of no promo sales", non_promo_sales.shape)
    print("# of promo sales", promo_sales.shape)

    return promo_sales, non_promo_sales
    #Product got sold  on promo time and non-promo time

In [4]:
print("SALES A:")
promotions_a = all_promotions.iloc[:4].copy()
promo_sales_a, non_promo_sales_a = promo_and_non_promo(sales1a, promotions_a )
promo_weeks_a = [7, 12, 22, 26]
print('promo_weeks',promo_weeks_a)

all_products = set(np.arange(len(sales1a['ProductCode'].unique()))+1)
all_stores = set(np.arange(len(sales1a['StoreCode'].unique()))+1)
n_week_a = len(sales1a["Week"].unique())
n_product, n_store = len(all_products), len(all_stores)
#print(sales1a.groupby(['Time']).size())

SALES A:
- 1873618 rows, 4 columns
- number of stores: 340, number of product: 317

True
True
hoop
Promotion Dates
 week  week StartDate EndDate
    7     8   Tuesday Tuesday
   11    12    Sunday  Sunday
   21    23    Sunday  Monday
   25    26    Sunday  Sunday

Promo vs Non Promo Time
# of no promo sales (1562695, 7)
# of promo sales (310923, 7)
promo_weeks [7, 12, 22, 26]


In [5]:
print("SALES A:")
promotions_b = pd.DataFrame([["Promo5","9/1/2015","9/6/2015"]], columns= ['Period','StartDate','EndDate'])

promo_sales_b, non_promo_sales_b = promo_and_non_promo(sales1b, promotions_b )
promo_weeks_b = [36]
print('promo_weeks',promo_weeks_b)

all_products_b = set(np.arange(len(sales1b['ProductCode'].unique()))+1)
all_stores_b = set(np.arange(len(sales1b['StoreCode'].unique()))+1)
n_week_b = len(sales1b["Week"].unique())
n_product_b, n_store_b = len(all_products_b), len(all_stores_b)


SALES A:
- 1028121 rows, 4 columns
- number of stores: 328, number of product: 302

True
True
hoop
Promotion Dates
 week  week StartDate EndDate
   36    36   Tuesday  Sunday

Promo vs Non Promo Time
# of no promo sales (986646, 7)
# of promo sales (41475, 7)
promo_weeks [36]


In [6]:
'''Calculatin pxs avg sale for promo and non promo times then combine these with features'''
def create_features(promo_sales, non_promo_sales, promo_week,p_time):
    
    np_pxs_weekly = non_promo_sales[(non_promo_sales.Week >= promo_week-3) & (non_promo_sales.Week <= promo_week + 3)]
    np_pxs_avg = np_pxs_weekly.groupby(['ProductCode', 'StoreCode'])['SalesQuantity'].sum()
                                               
    p_pxs_avg = promo_sales[promo_sales.Time == p_time].groupby(['ProductCode', 'StoreCode'])['SalesQuantity'].sum()

    missing_np_pxs = set(np_pxs_avg.index).difference(set(p_pxs_avg.index))
    missing_p_pxs = set(p_pxs_avg.index).difference(set(np_pxs_avg.index))
    #create series
    missing_np_pxs = pd.Series(0,index=missing_np_pxs, name='SalesQuantity')
    missing_np_pxs.index.names = ['ProductCode', 'StoreCode']

    missing_p_pxs = pd.Series(0, index=missing_p_pxs, name='SalesQuantity')
    missing_p_pxs.index.names = ['ProductCode', 'StoreCode']

    #add the missings
    np_pxs_avg = pd.concat([np_pxs_avg, missing_p_pxs])
    np_pxs_avg = np_pxs_avg.sort_index()
    np_pxs_avg.name = "pxs_Non_Promo_SalesQuantity"
    
    p_pxs_avg = pd.concat([p_pxs_avg, missing_np_pxs])
    p_pxs_avg = p_pxs_avg.sort_index()
    p_pxs_avg.name = "pxs_Promo_SalesQuantity"

    #pxs impact calculation -- which will be the value for the model to predict
    
    f_pxs = features.loc[np_pxs_avg.index]
    print(np_pxs_avg.shape[0]==p_pxs_avg.shape[0]==f_pxs.shape[0])

    X = pd.concat([f_pxs, np_pxs_avg],axis=1)
    Y = p_pxs_avg
    print(X.shape[0]==Y.shape[0])

    return X, Y

# pxs 3 weeks + - non promo avg | store_type | product_type | Product Group1 | Product Group2

In [7]:

data_X = []
data_y = []
for p_week, p_time in zip(promo_weeks_a,promotions_a.Period):
    print(p_time)
    X, y = create_features(promo_sales_a, non_promo_sales_a,p_week,p_time)
    data_X.append(X)
    data_y.append(y)

X_a = pd.concat(data_X,axis=0)
y_a = pd.concat(data_y,axis=0)


print("Promo5")
X_b, y_b = create_features(promo_sales_b, non_promo_sales_b, 36,"Promo5")


# Load your dataset

# One-Hot Encoding
X_a_encoded = pd.get_dummies(X_a, columns=['P_type', 'S_type','ProductGroup1','ProductGroup2'])
X_b_encoded = pd.get_dummies(X_b, columns=['P_type', 'S_type','ProductGroup1','ProductGroup2'])

# Splitting data into features and target
X = X_a_encoded.values
y = y_a.values

X_test = X_b_encoded.values
y_test = y_b.values

Promo1
True
True
Promo2
True
True
Promo3
True
True
Promo4
True
True
Promo5
True
True


In [8]:
scoring_metrics = ['neg_mean_squared_error', 'r2']

for depth in [3, 5, 8, 10, 12, 15, 20, 25]:
    for n_tree in [3, 5, 10, 15, 20]:
        print("\n")
        print("max_depth:", depth, "n_tree:", n_tree)
        reg = RandomForestRegressor(max_depth=depth, n_estimators=n_tree, random_state=0)
        cv_results = cross_validate(reg, X, y, cv=5, scoring=scoring_metrics)
        print("Val MSE Scores:", -cv_results['test_neg_mean_squared_error'], "mean:", -cv_results['test_neg_mean_squared_error'].mean())
        print("Val R2 Scores:", cv_results['test_r2'], "mean:", cv_results['test_r2'].mean())


# # Split into train and test sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train a Random Forest classifier
# reg = RandomForestRegressor(max_depth=10, random_state=0)

# reg.fit(X_train, y_train)

# # Make predictions on the test set
# val_pred = reg.predict(X_val)
# test_pred = reg.predict(X_test)

# # Evaluate the model
# r_score_val = reg.score(X_val, y_val)
# print(f"validation R Square: {r_score_val:.2f}")

# score_val = mean_squared_error(y_val, val_pred)
# print(f"validation mean_squared_error score: {score_val:.2f}")


# r_score_test = reg.score(X_test, y_test)
# print(f"test R Square: {r_score_test:.2f}")

# score_test = mean_squared_error(y_test, test_pred)
# print(f"test mean_squared_error score: {score_test:.2f}")



max_depth: 3 n_tree: 3
Val MSE Scores: [56.06328033 23.01624079 74.58520667 61.55302168 52.69070733] mean: 53.5816913599884
Val R2 Scores: [0.571922   0.53903188 0.53239993 0.5664058  0.56538575] mean: 0.5550290722400508


max_depth: 3 n_tree: 5
Val MSE Scores: [55.57816258 23.04055684 74.95630311 60.83025332 52.10807975] mean: 53.30267111742535
Val R2 Scores: [0.57562617 0.53854488 0.5300734  0.57149715 0.5701915 ] mean: 0.5571866214143558


max_depth: 3 n_tree: 10
Val MSE Scores: [55.06513451 23.04862944 74.7712571  60.37192269 51.95848753] mean: 53.04308625660663
Val R2 Scores: [0.57954346 0.5383832  0.53123351 0.57472574 0.5714254 ] mean: 0.5590622643180174


max_depth: 3 n_tree: 15
Val MSE Scores: [55.20816166 22.8707195  74.5155174  60.45485495 51.99655997] mean: 53.00916269622386
Val R2 Scores: [0.57845136 0.54194638 0.53283683 0.57414155 0.57111136] mean: 0.5596974959373874


max_depth: 3 n_tree: 20
Val MSE Scores: [55.06361896 22.99762133 74.49417948 60.36400932 52.02613807]

In [10]:


model = LinearRegression()

# Perform 5-fold cross-validation and evaluate accuracy
scoring_metrics = ['neg_mean_squared_error', 'r2']

cv_results = cross_validate(model, X, y, cv=5, scoring=scoring_metrics)

print("Val MSE Scores:", -cv_results['test_neg_mean_squared_error'], "mean:", -cv_results['test_neg_mean_squared_error'].mean())
print("Val R2 Scores:", cv_results['test_r2'], "mean:", cv_results['test_r2'].mean() )

# print("Linear Regression")
# reg = LinearRegression().fit(X_train, y_train)
# print("Score", reg.score(X_train, y_train))
# print("Test Score", reg.score(X_val, y_val))

Val MSE Scores: [54.20329867 22.89393706 70.48433444 58.12757221 50.10241883] mean: 51.16231223999711
Val R2 Scores: [0.58612412 0.54148138 0.55810969 0.59053549 0.586735  ] mean: 0.5725971340756646


In [13]:
#Test Result

# Picked the best performing according to the Cross Validation
reg = RandomForestRegressor(max_depth=5,n_estimators=5, random_state=0)
reg.fit(X, y)


test_pred = reg.predict(X_test)

# Evaluate the model

r_score_test = reg.score(X_test, y_test)
print(f"test R Square: {r_score_test:.2f}")

score_test = mean_squared_error(y_test, test_pred)
print(f"test mean_squared_error score: {score_test:.2f}")

test R Square: 0.27
test mean_squared_error score: 33.92


In [15]:
display(X_a)

P_type ProductGroup1  ProductGroup2  S_type  \
ProductCode StoreCode                                                
1           1          medium             A              5  medium   
            2          medium             A              5    fast   
            3          medium             A              5    slow   
            5          medium             A              5    slow   
            6          medium             A              5  medium   
...                       ...           ...            ...     ...   
317         331          slow             J              8    fast   
            333          slow             J              8    fast   
            334          slow             J              8  medium   
            335          slow             J              8  medium   
            338          slow             J              8  medium   

                       pxs_Non_Promo_SalesQuantity  
ProductCode StoreCode                               
1           1                                    2  
            2                                   21  
            3                                   24  
            5                                    2  
            6                                    8  
...                                            ...  
317         331                                  2  
            333                                  2  
            334                                  1  
            335                                  0  
            338                                  1  

[258712 rows x 5 columns]